# Import

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as sm 
pd.set_option('display.max_colwidth', -1)
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [10, 5]
from xgboost.sklearn import XGBClassifier 
import xgboost as xgb

['test_images', 'breed_labels.csv', '.DS_Store', 'test', 'train_metadata', 'color_labels.csv', 'test_sentiment', 'test_metadata', 'train_sentiment', 'train', 'train_images', 'state_labels.csv']


## Cassic data

In [2]:
train = pd.read_csv('../input/train/train.csv')
train.shape


train_y = train['AdoptionSpeed']
train.drop(columns = 'AdoptionSpeed', inplace=True)

train_y.shape

train.shape

test = pd.read_csv('../input/test/test.csv')
test.shape

df = pd.concat([train, test], ignore_index=True, sort=False)

df.shape

df.Description = df.Description.astype('str')

df['DescLen'] = df.Description.astype(str).fillna('').apply(len)

df['NameLen'] = np.log1p(df['Name'].astype(str).fillna('').apply(len))

In [3]:
def modelfit(alg, dtrain, sel_cols,target, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[sel_cols].values, label=target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds, early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[sel_cols], target.values)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[sel_cols])
    dtrain_predprob = alg.predict_proba(dtrain[sel_cols])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Quadratic gamma : %.4g" % quadratic_weighted_kappa(target.values, dtrain_predictions)   )           
    feat_imp = pd.Series(model.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    return(alg)

In [4]:
one_hot_cols=['Gender', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized','Health']

df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)



cols_log = ['Age', 'Quantity', 'Fee', 'PhotoAmt', 'VideoAmt', 'DescLen', 'NameLen']

df[cols_log] = df[cols_log].apply(np.log1p)

In [5]:
df['Breed1Col1'] = df['Breed1'] * df['Color1']
df['Breed1Col2'] = df['Breed1'] * df['Color2']
df['Breed2Col1'] = df['Breed2'] * df['Color1']
df['Breed2Col2'] = df['Breed2'] * df['Color2']

In [18]:
sel_cols = [
    'Type', 'Age', 'Breed1', 'Breed2', 'Color1', 'Color2', 'Color3',
    'Quantity', 'Fee', 'State', 'DescLen', 'NameLen', 'Gender_2', 'Gender_3',
    'MaturitySize_2', 'MaturitySize_3', 'MaturitySize_4', 'FurLength_2',
    'FurLength_3', 'Vaccinated_2', 'Vaccinated_3', 'Dewormed_2', 'Dewormed_3',
    'Sterilized_2', 'Sterilized_3', 'Health_2', 'Health_3', 'VideoAmt',
    'PhotoAmt', 'Breed1Col1',  'Breed1Col2', 'Breed2Col2', 'Breed2Col1'
]

df_ml = df[sel_cols]
df_ml.State = df_ml.State.astype(int)

df_ml.State = df_ml.State.astype(int)

train_ml = df_ml.iloc[0:14993,]
test_ml = df_ml.iloc[14993:,]

#xgb1 = xgb.XGBClassifier(n_estimators=500, nthread=-1, max_depth=8, learning_rate=0.015, num_class =5, objective='multi:softprob')

model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.015, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=4, nthread=-1, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
model.fit(train_ml.as_matrix(),train_y.values)

Exception ignored in: <function DMatrix.__del__ at 0x1213c21e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/xgboost/core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/usr/local/Cellar/ipython/7.1.1/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.015, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=4, nthread=-1, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [19]:

pred = model.predict(test_ml.as_matrix())

/usr/local/Cellar/ipython/7.1.1/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [ ]:
#pd.Series(pred).plot.hist()

# Submission

In [ ]:
sample = pd.read_csv('../input/test/sample_submission.csv')
sample.AdoptionSpeed = pred
sample.to_csv('submission.csv', index=False)
